# NHCI BAMS Summary Statistics

In [1]:
import os
import pickle
from typing import List, Tuple, Optional

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy

from src import configure, plotting, utilities

In [2]:
config = configure.get_config()
configure.configure_ipython()
plotting.configure_figures()

## NHCI IDs

In [3]:
ian_drifter_ids = {
    'spotter': [
        'SPOT-30068D',
        'SPOT-30097D',
        'SPOT-30096D',
        'SPOT-30104D',
        'SPOT-30024D',
        'SPOT-30065D',
    ],
    'dwsd': [
        '300534061617480',
        '300534061618130',
        '300534061800920',
        '300534061618410',
        '300534061550970',
    ],
    'microswift': [
        '019',
        # '034',
        '057',
        # '061',
    ],
}

lee_drifter_ids = {
    'spotter': [
        # 'SPOT-31232D',  # c?
        # 'SPOT-31237D',  # c?
        # 'SPOT-31233D',  # c?
        'SPOT-31232C',
    ],
    'dwsd': [
        # '300534061807800',
        '300534061901680',
        '300534061901630',
        '300534061901660',
    ],
    'microswift': [   #TODO: require variance correction
        '041',
        '050',
        '059',
        '065',
    ],
}


idalia_drifter_ids = {
    'spotter': [
        'SPOT-30025D',
        'SPOT-30052D',
        'SPOT-30055D',
        'SPOT-30061D',
        'SPOT-30066D',
        'SPOT-30095D',
        'SPOT-30101D',
        'SPOT-30102D',
        'SPOT-30103D',
        'SPOT-30164D',
    ],
    'dwsd': [
        '300534064800060',
        '300534064800070',
        '300534064800090',
        '300534064800250',
    ],
    'microswift': [
        '029',
        '037',
        '046',
        '048',
    ],
}

francine_drifter_ids = {
    'spotter': [
        'SPOT-31989C',
        'SPOT-31993C',
        'SPOT-31984C',
        'SPOT-31994C',
        'SPOT-31983C',
    ],
    'dwsd': [
        '300534061904690',
        '300534061904700',
        '300534061904680',
    ],
    'microswift': [
        '098',
        '099',
        '101',
    ],
}

helene_drifter_ids = {
    'spotter': [
        'SPOT-31990C',
        'SPOT-31840C',
        'SPOT-31832C',
        'SPOT-31838C',
        'SPOT-31837C',
        'SPOT-31987C',
        'SPOT-31834C',
        'SPOT-31831C',
        'SPOT-31836C',
        'SPOT-31839C',
        'SPOT-31835C',
        'SPOT-31988C',
        'SPOT-31995C',
    ],
    'dwsd': [
        '300534061904750',
        '300534061904800',
        '300534061904660',
        '300534061905080',
        '300534061904760',
        '300534061904670',
    ],
}

milton_drifter_ids = {
    'spotter': [
        'SPOT-31234C',
        'SPOT-31243C',
        'SPOT-31245C',
        'SPOT-31829C',
        'SPOT-31830C',
        'SPOT-31833C',
        'SPOT-31986C',
        'SPOT-31992C',
        'SPOT-31840C',  # helene
        'SPOT-31832C',
        'SPOT-31838C',
        'SPOT-31837C',
        'SPOT-31987C',
        'SPOT-31834C',
        'SPOT-31831C',
        'SPOT-31836C',
        'SPOT-31839C',
        'SPOT-31835C',
        'SPOT-31988C',
        'SPOT-31995C',
    ],
    'dwsd': [
        '300534061905440',
        '300534061904650',
        '300534061905090',
        '300534061905110',
        '300534061905650',
        '300534061904640',
        '300534061905120',
        '300534061905600',
        '300534061904750',  # helene
        '300534061904800',
        '300534061904660',
        '300534061905080',
        '300534061904760',
        '300534061904670',
    ],
    'microswift': [
        '078',
        '079',
        # '103',
        # '106',  # helene NOAA
    ],
}

## Data

### Drifters

In [4]:
def read_drifter_dataset(path: str) -> pd.DataFrame:
    with open(path, 'rb') as handle:
        drifter_data = pickle.load(handle)

    # Concatenate the individual drifter DataFrames by type
    microswift_df = concatenate_drifters(drifter_data['microswift'])
    spotter_df = concatenate_drifters(drifter_data['spotter'])
    dwsd_df = concatenate_drifters(drifter_data['dwsd'])

    # Create a drifter type column
    microswift_df['drifter_type'] = 'microswift'
    spotter_df['drifter_type'] = 'spotter'
    dwsd_df['drifter_type'] = 'dwsd'

    # Combine all drifters into a single DataFrame.
    drifter_df = (pd.concat([microswift_df, spotter_df, dwsd_df])
                .sort_index(level=['id', 'time'], ascending=True))
    # .loc[(slice(None), time_slice), :]
    return drifter_df


def concatenate_drifters(drifter_dict: dict) -> pd.DataFrame:
    """
    Concatenate a dictionary of individual drifter DataFrames into a single,
    multi-index DataFrame.  Drop the observations that do not contain waves
    (remove off-hour pressure and temperature observations).

    Args:
        drifter_dict (dict): individual drifter DataFrames keyed by id.

    Returns:
        DataFrame: concatenated drifters
    """
    drifter_df = (
        pd.concat(drifter_dict, names=['id', 'time'])
        .dropna(subset='energy_density')
    )
    return drifter_df

In [5]:
ian_drifter_df = read_drifter_dataset(config['dir']['ian_drifter_data'])
lee_drifter_df = read_drifter_dataset(config['dir']['lee_drifter_data'])
idalia_drifter_df = read_drifter_dataset(config['dir']['idalia_drifter_data'])
francine_drifter_df = read_drifter_dataset(config['dir']['francine_drifter_data'])
helene_drifter_df = read_drifter_dataset(config['dir']['helene_drifter_data'])
milton_drifter_df = read_drifter_dataset(config['dir']['milton_drifter_data'])

### IBTrACS

In [6]:
IBTRACS_PATH = config['dir']['ibtracs']
storm_names = ["IAN", "IDALIA", "LEE", "FRANCINE", "HELENE", "MILTON"]
ibtracs_df = pd.read_csv(IBTRACS_PATH, low_memory=False)
ibtracs_df = (ibtracs_df
    .query(f'NAME == {storm_names}')
    # .query('SEASON == "2023"')
    # .query('BASIN == "NA"')
    .query('SEASON == ["2022", "2023", "2024"]')
    .assign(ISO_TIME = lambda df: pd.to_datetime(df['ISO_TIME'], utc=True))
    .set_index(['NAME', 'ISO_TIME'], drop=True)
    # .set_index('ISO_TIME', drop=True)
    .assign(LAT = lambda df: df['LAT'].astype(np.float64))
    .assign(LON = lambda df: df['LON'].astype(np.float64))
    .assign(STORM_DIR = lambda df: df['STORM_DIR'].astype(np.float64))
    .assign(USA_RMW = lambda df: df['USA_RMW'].astype(np.float64))
)

## Preprocessing

In [7]:
def interpolate_ibtracs(ibtracs_df, drifter_df):
    coords = ibtracs_df[['LON', 'LAT']]
    direction = ibtracs_df['STORM_DIR']
    direction_unwrap = np.unwrap(direction, period=360)
    storm_speed = ibtracs_df['STORM_SPEED'] # kt
    radius_max_wind = ibtracs_df['USA_RMW'] # nm
    max_wind_speed = ibtracs_df['USA_WIND']# kt

    times = pd.to_numeric(ibtracs_df.index.tz_localize(None).to_numpy())
    interp1d_kwargs = dict(fill_value=np.nan, axis=0, bounds_error=False)
    coords_interpolator = scipy.interpolate.interp1d(times, coords, **interp1d_kwargs)
    direction_interpolator = scipy.interpolate.interp1d(times, direction_unwrap, **interp1d_kwargs)
    storm_speed_interpolator = scipy.interpolate.interp1d(times, storm_speed, **interp1d_kwargs)
    radius_max_wind_interpolator = scipy.interpolate.interp1d(times, radius_max_wind, **interp1d_kwargs)
    max_wind_speed_interpolator = scipy.interpolate.interp1d(times, max_wind_speed, **interp1d_kwargs)

    storm_positions = get_time_interpolation(coords_interpolator, drifter_df.index.get_level_values(level='time'))
    storm_headings = get_time_interpolation(direction_interpolator, drifter_df.index.get_level_values(level='time'))
    storm_speeds = get_time_interpolation(storm_speed_interpolator, drifter_df.index.get_level_values(level='time'))
    storm_radius_max_winds = get_time_interpolation(radius_max_wind_interpolator, drifter_df.index.get_level_values(level='time'))
    storm_max_wind_speeds = get_time_interpolation(max_wind_speed_interpolator, drifter_df.index.get_level_values(level='time'))

    return drifter_df.assign(
        storm_longitude=storm_positions[0],
        storm_latitude=storm_positions[1],
        storm_heading=storm_headings % 360,
        storm_speed_kt=storm_speeds,
        storm_radius_max_wind_nmi=storm_radius_max_winds,
        storm_max_wind_speed_kt=storm_max_wind_speeds,
    )

def get_time_interpolation(
    interpolator: scipy.interpolate.interp1d,
    query_times: pd.DatetimeIndex,
) -> Tuple[np.ndarray, ...]:
    """ Interpolate a time series of data at query times (datetimes)."""
    numeric_query_times = pd.to_numeric(query_times)
    return interpolator(numeric_query_times).T



In [8]:
ian_drifter_df = interpolate_ibtracs(ibtracs_df.loc['IAN'], ian_drifter_df)
idalia_drifter_df = interpolate_ibtracs(ibtracs_df.loc['IDALIA'], idalia_drifter_df)
lee_drifter_df = interpolate_ibtracs(ibtracs_df.loc['LEE'], lee_drifter_df)
francine_drifter_df = interpolate_ibtracs(ibtracs_df.loc['FRANCINE'], francine_drifter_df)
helene_drifter_df = interpolate_ibtracs(ibtracs_df.loc['HELENE'], helene_drifter_df)
milton_drifter_df = interpolate_ibtracs(ibtracs_df.loc['MILTON'], milton_drifter_df)

In [9]:
def great_circle_pairwise(
    longitude_a: np.ndarray,
    latitude_a: np.ndarray,
    longitude_b: np.ndarray,
    latitude_b: np.ndarray,
    earth_radius: float = 6378.137,
    mod_bearing: bool = True
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computes the great circle distance (km) and true fore bearing (deg) between
    pairs of observations in input arrays `longitude_a` and `longitude_b` and
    `latitude_a` and `latitude_b`.

    For two longitude and latitude pairs, the great circle distance is the
    shortest distance between the two points along the Earth's surface. This
    distance is calculated using the Haversine formula. The instances in
    `longitude_a` and `latitude_a` are designated as point `a`; the instances
    in `longitude_b` and `latitude_b` then form point `b`. The true fore
    bearing is the bearing, measured from true north, of `b` as seen from `a`.

    Note:
        When given `latitude_a/b` and `longitude_a/b` of shape (n,), n > 1,
        the great circle distance and fore bearing will be calculated between
        `a` and `b` entries such that the returned arrays will be of shape
        (n,). To compute the great circle distance and bearings between
        adjacent coordinates of single longitude and latitude arrays (i.e.,
        along a trajectory), use `great_circle_pathwise`.

    Args:
        longitude_a (np.array): of shape (n,) in units of decimal degrees
        latitude (np.array): of shape (n,) in units of decimal degrees
        earth_radius (float, optional): earth's radius in units of km. Defaults to 6378.137 km (WGS-84)
        mod_bearing (bool, optional): return bearings modulo 360 deg. Defaults to True.

    Returns:
        Tuple[np.array, np.array]: great circle distances (in km) and true fore
        bearings between adjacent longitude and latitude pairs; shape (n,)
    """
    # Convert decimal degrees to radians
    longitude_a_rad, latitude_a_rad = map(np.radians, [longitude_a, latitude_a])
    longitude_b_rad, latitude_b_rad = map(np.radians, [longitude_b, latitude_b])

    # Difference longitude and latitude
    longitude_difference = longitude_b_rad - longitude_a_rad
    latitude_difference = latitude_b_rad - latitude_a_rad

    # Haversine formula
    a_1 = np.sin(latitude_difference / 2) ** 2
    a_2 = np.cos(latitude_a_rad)
    a_3 = np.cos(latitude_b_rad)
    a_4 = np.sin(longitude_difference / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a_1 + a_2 * a_3 * a_4))
    distance_km = earth_radius * c

    # True bearing
    bearing_num = np.cos(latitude_b_rad) * np.sin(-longitude_difference)
    bearing_den_1 = np.cos(latitude_a_rad) * np.sin(latitude_b_rad)
    bearing_den_2 = - np.sin(latitude_a_rad) * np.cos(latitude_b_rad) * np.cos(longitude_difference)
    bearing_deg = -np.degrees(np.arctan2(bearing_num, bearing_den_1 + bearing_den_2))

    if mod_bearing:
        bearing_deg = bearing_deg % 360

    return distance_km, bearing_deg

In [10]:
def calculate_storm_distance(drifter_df):
    storm_distance, storm_back_bearing = great_circle_pairwise(
        longitude_a=drifter_df['storm_longitude'],
        latitude_a=drifter_df['storm_latitude'],
        longitude_b=drifter_df['longitude'],
        latitude_b=drifter_df['latitude'],
    )
    # drifter_df['storm_radius_max_wind_km'] = utilities.nmi_to_km(drifter_df['storm_radius_max_wind_nmi'])
    storm_distance_rmw_norm = storm_distance / utilities.nmi_to_km(drifter_df['storm_radius_max_wind_nmi'])
    return drifter_df.assign(
        storm_distance=storm_distance,
        storm_back_bearing=storm_back_bearing,
        storm_distance_rmw_norm=storm_distance_rmw_norm,
    )

In [11]:
ian_drifter_df = calculate_storm_distance(ian_drifter_df)
idalia_drifter_df = calculate_storm_distance(idalia_drifter_df)
lee_drifter_df = calculate_storm_distance(lee_drifter_df)
francine_drifter_df = calculate_storm_distance(francine_drifter_df)
helene_drifter_df = calculate_storm_distance(helene_drifter_df)
milton_drifter_df = calculate_storm_distance(milton_drifter_df)

Create a concatenated dataset of NHCI-only drifters.

In [12]:
def merge_drifter_ids(drifter_id_dict: dict) -> List:
    drifter_ids = []
    for drifter_type, drifter_id in drifter_id_dict.items():
        drifter_ids += drifter_id
    return drifter_ids

nhci_drifter_concat_df = pd.concat(
    objs = [
        ian_drifter_df.loc[merge_drifter_ids(ian_drifter_ids)],
        idalia_drifter_df.loc[merge_drifter_ids(idalia_drifter_ids)],
        lee_drifter_df.loc[merge_drifter_ids(lee_drifter_ids)],
        francine_drifter_df.loc[merge_drifter_ids(francine_drifter_ids)],
        helene_drifter_df.loc[merge_drifter_ids(helene_drifter_ids)],
        milton_drifter_df.loc[merge_drifter_ids(milton_drifter_ids)],
    ],
    keys=['2022_IAN', '2023_IDALIA', '2023_LEE', '2024_FRANCINE', '2024_HELENE', '2024_MILTON'],
    names=['storm', 'id', 'time'],
)

## Summary Statistics

storms sampled, total number of buoys deployed, hours of data collected, data within radius of max winds, and anything else you find informative.

In [13]:
nhci_drifter_concat_df.groupby('storm')['storm_distance_rmw_norm'].describe()


,count,mean,std,min,25%,50%,75%,max
storm,,,,,,,,
2022_IAN,1480.0,10.861139,5.562839,0.857143,6.915938,9.395207,13.610326,27.151272
2023_IDALIA,1443.0,12.194344,5.275346,0.136586,8.464604,10.997693,15.233555,27.360104
2023_LEE,847.0,4.860585,2.946404,1.492798,2.638641,3.880869,6.511470,34.673172
2024_FRANCINE,904.0,5.792501,2.601189,0.297184,3.543691,5.608390,7.216318,11.588058
2024_HELENE,2038.0,8.328917,5.923338,0.359057,3.781513,6.374423,12.003328,23.990032
2024_MILTON,2218.0,17.508661,12.478252,0.302693,6.044617,13.669223,28.836766,43.633108


### Stats by number of drifters

Number of drifters in each storm:

In [14]:
summary_stats_dict = {}

In [15]:
# TODO: this is complicated by the fact that there are non-NOPP drifters in dataset.

summary_stats_dict['total_number_of_drifters'] = (
    nhci_drifter_concat_df.reset_index('id').groupby('storm')['id'].nunique()
)
print(summary_stats_dict['total_number_of_drifters'])


storm
2022_IAN         13
2023_IDALIA      18
2023_LEE          8
2024_FRANCINE    11
2024_HELENE      19
2024_MILTON      36
Name: id, dtype: int64


Number of drifters within *n* km:

In [16]:
# TODO: for dist in ...
query = 'storm_distance < 500'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).reset_index('id').groupby('storm')['id'].nunique()
)
print(summary_stats_dict[query])

query = 'storm_distance < 100'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).reset_index('id').groupby('storm')['id'].nunique()
)
print(summary_stats_dict[query])

query = 'storm_distance < 50'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).reset_index('id').groupby('storm')['id'].nunique()
)
print(summary_stats_dict[query])

storm
2022_IAN         13
2023_IDALIA      18
2023_LEE          7
2024_FRANCINE    11
2024_HELENE      19
2024_MILTON      31
Name: id, dtype: int64
storm
2022_IAN          2
2023_IDALIA      11
2024_FRANCINE     7
2024_HELENE      10
2024_MILTON      10
Name: id, dtype: int64
storm
2022_IAN         1
2023_IDALIA      7
2024_FRANCINE    3
2024_HELENE      5
2024_MILTON      6
Name: id, dtype: int64


Number of drifters within *n* RMW:

In [17]:
query = 'storm_distance_rmw_norm <= 10'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).reset_index('id').groupby('storm')['id'].nunique()
)
print(summary_stats_dict[query])

query = 'storm_distance_rmw_norm <= 5'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).reset_index('id').groupby('storm')['id'].nunique()
)
print(summary_stats_dict[query])

query = 'storm_distance_rmw_norm <= 1'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).reset_index('id').groupby('storm')['id'].nunique()
)
print(summary_stats_dict[query])

storm
2022_IAN         13
2023_IDALIA      18
2023_LEE          8
2024_FRANCINE    11
2024_HELENE      19
2024_MILTON      35
Name: id, dtype: int64
storm
2022_IAN          9
2023_IDALIA      11
2023_LEE          8
2024_FRANCINE    11
2024_HELENE      18
2024_MILTON      20
Name: id, dtype: int64
storm
2022_IAN         1
2023_IDALIA      3
2024_FRANCINE    2
2024_HELENE      5
2024_MILTON      2
Name: id, dtype: int64


In [18]:
summary_stats_df = pd.concat(summary_stats_dict, axis=1).fillna(0).astype(int)
summary_stats_df.to_csv('nhci_stats_by_number_of_buoys.csv')
summary_stats_df

,total_number_of_drifters,storm_distance < 500,storm_distance < 100,storm_distance < 50,storm_distance_rmw_norm <= 10,storm_distance_rmw_norm <= 5,storm_distance_rmw_norm <= 1
storm,,,,,,,
2022_IAN,13,13,2,1,13,9,1
2023_IDALIA,18,18,11,7,18,11,3
2023_LEE,8,7,0,0,8,8,0
2024_FRANCINE,11,11,7,3,11,11,2
2024_HELENE,19,19,10,5,19,18,5
2024_MILTON,36,31,10,6,35,20,2


In [19]:
summary_stats_df.sum(axis=0)

total_number_of_drifters         105
storm_distance < 500              99
storm_distance < 100              40
storm_distance < 50               22
storm_distance_rmw_norm <= 10    104
storm_distance_rmw_norm <= 5      77
storm_distance_rmw_norm <= 1      13
dtype: int64

### Stats by number of observations

Number of observations in each storm:

In [20]:
summary_stats_dict = {}

In [21]:
summary_stats_dict['total_number_of_observations'] = (
    nhci_drifter_concat_df.groupby('storm').size()
)
print(summary_stats_dict['total_number_of_observations'])


storm
2022_IAN         1570
2023_IDALIA      1443
2023_LEE          847
2024_FRANCINE     904
2024_HELENE      2326
2024_MILTON      2795
dtype: int64


Measurements within *n* km:

In [22]:
query = 'storm_distance < 500'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).groupby('storm').size()
)
print(summary_stats_dict[query])

query = 'storm_distance < 100'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).groupby('storm').size()
)
print(summary_stats_dict[query])

query = 'storm_distance < 50'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).groupby('storm').size()
)
print(summary_stats_dict[query])

storm
2022_IAN          881
2023_IDALIA       750
2023_LEE          218
2024_FRANCINE     588
2024_HELENE       526
2024_MILTON      1181
dtype: int64
storm
2022_IAN         24
2023_IDALIA      73
2024_FRANCINE    33
2024_HELENE      56
2024_MILTON      97
dtype: int64
storm
2022_IAN          4
2023_IDALIA      25
2024_FRANCINE     6
2024_HELENE      13
2024_MILTON      27
dtype: int64


Measurements within *n* RMW:

In [23]:
query = 'storm_distance_rmw_norm <= 10'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).groupby('storm').size()
)
print(summary_stats_dict[query])

query = 'storm_distance_rmw_norm <= 5'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).groupby('storm').size()
)
print(summary_stats_dict[query])

query = 'storm_distance_rmw_norm <= 1'
summary_stats_dict[query] = (
    nhci_drifter_concat_df.query(query).groupby('storm').size()
)
print(summary_stats_dict[query])


storm
2022_IAN          823
2023_IDALIA       595
2023_LEE          786
2024_FRANCINE     808
2024_HELENE      1466
2024_MILTON       954
dtype: int64
storm
2022_IAN         155
2023_IDALIA       66
2023_LEE         533
2024_FRANCINE    398
2024_HELENE      656
2024_MILTON      347
dtype: int64
storm
2022_IAN         2
2023_IDALIA      4
2024_FRANCINE    3
2024_HELENE      9
2024_MILTON      4
dtype: int64


In [32]:
summary_stats_df = pd.concat(summary_stats_dict, axis=1).fillna(0).astype(int)
summary_stats_df.to_csv('nhci_stats_by_number_of_obs.csv')
summary_stats_df

,total_number_of_observations,storm_distance < 500,storm_distance < 100,storm_distance < 50,storm_distance_rmw_norm <= 10,storm_distance_rmw_norm <= 5,storm_distance_rmw_norm <= 1
storm,,,,,,,
2022_IAN,1570,881,24,4,823,155,2
2023_IDALIA,1443,750,73,25,595,66,4
2023_LEE,847,218,0,0,786,533,0
2024_FRANCINE,904,588,33,6,808,398,3
2024_HELENE,2326,526,56,13,1466,656,9
2024_MILTON,2795,1181,97,27,954,347,4


In [25]:
summary_stats_df.sum(axis=0)

total_number_of_observations     9885
storm_distance < 500             4144
storm_distance < 100              283
storm_distance < 50                75
storm_distance_rmw_norm <= 10    5432
storm_distance_rmw_norm <= 5     2155
storm_distance_rmw_norm <= 1       22
dtype: int64